d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 1200px">
</div>

# Transformations & Actions

**Technical Accomplishments:**
* Review the Lazy vs. Eager design
* Quick review of Transformations
* Quick review of Actions
* Introduce the Catalyst Optimizer
* Wide vs. Narrow Transformations

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Getting Started

Run the following cell to configure our "classroom."

In [0]:
%run "../Includes/Classroom-Setup"

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Laziness By Design

Fundamental to Apache Spark are the notions that
* Transformations are **LAZY**
* Actions are **EAGER**

We see this play out when we run multiple transformations back-to-back, and no job is triggered:

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType

schema = StructType(
  [
    StructField("project", StringType(), False),
    StructField("article", StringType(), False),
    StructField("requests", IntegerType(), False),
    StructField("bytes_served", LongType(), False)
  ]
)

parquetFile = "dbfs:/mnt/training/wikipedia/pagecounts/staging_parquet_en_only_clean/"

topTenDF = (spark                                          # Our SparkSession & Entry Point
  .read                                                    # DataFrameReader
  .schema(schema)                                          # DataFrameReader (config)
  .parquet(parquetFile)                                    # Transformation (initial)
  .where( "project = 'en'" )                               # Transformation
  .drop("bytes_served")                                    # Transformation
  .filter( col("article") != "Main_Page")                  # Transformation
  .filter( col("article") != "-")                          # Transformation
  .filter( col("article").startswith("Special:") == False) # Transformation
)
print(topTenDF) # Python hack to print the data type

In [0]:
display(topTenDF)

There is one exception to this.

When you are reading data, Apache Spark needs to know the schema of the `DataFrame`.

In this case, the schema was specified.

However, if the `DataFrameReader` doesn't specify the schema, it will trigger a one-time job to examine the files on disk and determine the schema:

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType

# schema = StructType(
#   [
#     StructField("project", StringType(), False),
#     StructField("article", StringType(), False),
#     StructField("requests", IntegerType(), False),
#     StructField("bytes_served", LongType(), False)
#   ]
# )

parquetFile = "dbfs:/mnt/training/wikipedia/pagecounts/staging_parquet_en_only_clean/"

(spark                                                      # Our SparkSession & Entry Point
  .read                                                     # DataFrameReader
  #.schema(schema)                                          # DataFrameReader (config)
  .parquet(parquetFile)                                     # Transformation (initial)
  .where( "project = 'en'" )                                # Transformation
  .drop("bytes_served")                                     # Transformation
  .filter( col("article") != "Main_Page")                   # Transformation
  .filter( col("article") != "-")                           # Transformation
  .filter( col("article").startswith("Special:") == False)  # Transformation
)

Back to our instance of `topTenDF`...

With it, we can trigger a job by calling an action such as `count()`

In [0]:
topTenDF.count()

### Why is Laziness So Important?

This is a common pattern in functional programming as well as with Big Data specific languages.
* We see it in Scala as part of its core design.
* Java 8 introduced the concept with its Streams API.
* And many functional programming languages have similar APIs.

It has a number of benefits
* Not forced to load all data at step #1 
  * Technically impossible with **REALLY** large datasets.
* Easier to parallelize operations 
  * N different transformations can be processed on a single data element, on a single thread, on a single machine. 
* Most importantly, it allows the framework to automatically apply various optimizations

### Catalyst Optimizer

Because our API is declarative a large number of optimizations are available to us.

Some of the examples include:
  * Optimizing data type for storage
  * Rewriting queries for performance
  * Predicate push downs

![Catalyst](https://files.training.databricks.com/images/105/catalyst-diagram.png)

Additional Articles:
* <a href="https://databricks.com/session/deep-dive-into-catalyst-apache-spark-2-0s-optimizer" target="_blank">Deep Dive into Catalyst: Apache Spark 2.0's Optimizer</a>, Yin Huai's Spark Summit 2016 presentation.
* <a href="https://www.youtube.com/watch?v=6bCpISym_0w" target="_blank">Catalyst: A Functional Query Optimizer for Spark and Shark</a>, Michael Armbrust's presentation at ScalaDays 2016.
* <a href="https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html" target="_blank">Deep Dive into Spark SQL's Catalyst Optimizer</a>, Databricks Blog, April 13, 2015.
* <a href="http://people.csail.mit.edu/matei/papers/2015/sigmod_spark_sql.pdf" target="_blank">Spark SQL: Relational Data Processing in Spark</a>, Michael Armbrust, Reynold S. Xin, Cheng Lian, Yin Huai, Davies Liu, Joseph K. Bradley, Xiangrui Meng, Tomer Kaftan, Michael J. Franklin, Ali Ghodsi, Matei Zaharia,<br/>_Proceedings of the 2015 ACM SIGMOD International Conference on Management of Data_.

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Actions

Transformations always return a `DataFrame` (or in some cases, such as Scala & Java, `Dataset[Row]`).

In contrast, Actions either return a result or write to disc. For example:
* The number of records in the case of `count()` 
* An array of objects in the case of `collect()` or `take(n)`

We've seen a good number of the actions - most of them are listed below. 

For the complete list, one needs to review the API docs.

| Method | Return | Description |
|--------|--------|-------------|
| `collect()` | Collection | Returns an array that contains all of Rows in this Dataset. |
| `count()` | Long | Returns the number of rows in the Dataset. |
| `first()` | Row | Returns the first row. |
| `foreach(f)` | - | Applies a function f to all rows. |
| `foreachPartition(f)` | - | Applies a function f to each partition of this Dataset. |
| `head()` | Row | Returns the first row. |
| `reduce(f)` | Row | Reduces the elements of this Dataset using the specified binary function. |
| `show(..)` | - | Displays the top 20 rows of Dataset in a tabular form. |
| `take(n)` | Collection | Returns the first n rows in the Dataset. |
| `toLocalIterator()` | Iterator | Return an iterator that contains all of Rows in this Dataset. |

** *Note #1:* ** *There are some variations in the methods from language to language *

** *Note #2:* ** *The command `display(..)` is not included here because it's not part of the Spark API, even though it ultimately calls an action. *

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Transformations

Transformations have the following key characteristics:
* They eventually return another `DataFrame` (or in the case of Scala and Java, `Dataset[T]`).
* They are immutable - that is each instance of a `DataFrame` cannot be altered once it's instantiated.
  * This means other optimizations are possible - such as the use of shuffle files (to be discussed in detail later)
* Are classified as either a Wide or Narrow operation
* In Scala & Java come in two flavors: Typed & Untyped

Let's take a look at the API docs - in this case, looking at the Scala doc is a little easier to decipher.

** *Note:* ** The list of transformations varies significantly between each language.<br/>
Mostly because Java & Scala are statically typed languages compared Python & R which are dynamically typed.

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Typed vs. Untyped Transformations

The difference between **Typed** and **Untyped** transformations is specific to Scala and Java where Datasets can contain statically typed objects (e.g. `Dataset[Person]`).

A typed transformation does not change the columns and thus leaves the data types unchanged (e.g. `Dataset[T]`).  An untyped transformations can change the column names and therefore return a generic `DataFrame` (i.e. `Dataset[Row]`).

| Typed Transformations <br/> returns *Dataset[T]*  | Untyped Transformations <br/> returns *DataFrame* | 
|:----------------------------:|:-----------------------------:|
| `alias(..)`                  | `agg(..)`                     |
| `as(..)`                     | `apply(..)`                   |
| `coalesce(..)`               | `col(..)`                     |
| `distinct(..)`               | `crossJoin(..)`               |
| `dropDuplicates(..)`         | `cube(..)`                    |
| `except(..)`                 | `drop(..)`                    |
| `filter(..)`                 | `groupBy(..)`                 |
| `flatMap(..)`                | `join(..)`                    |
| `groupByKey(..)`             | `rollup(..)`                  | 
| `intersect(..)`              | `select(..)`                  |
| `limit(..)`                  | `selectExpr(..)`              |                    
| `map(..)`                    | `stat(..)`                    |                    
| `mapPartitions(..)`          | `withColumn(..)`              |                    
| `orderBy(..)`                | `withColumnRenamed(..)`       |                     
| `randomSplit(..)`            |                               |                     
| `repartition(..)`            |                               |                     
| `sample(..)`                 |                               |                     
| `sort(..)`                   |                               |                     
| `sortWithinPartitions(..)`   |                               |                     
| `union(..)`                  |                               |                     
| `where(..)`                  |                               |

-sandbox
##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Wide vs. Narrow Transformations

Regardless of language, transformations break down into two broad categories: wide and narrow.

**Narrow Transformations**: The data required to compute the records in a single partition reside in at most one partition of the parent RDD.

Examples include:
* `select(..)`  
* `filter(..)`  
* `coalesce()`  

<img src="https://files.training.databricks.com/images/105/transformations-narrow.png" alt="Narrow Transformations" style="height:300px"/>

<br/>

**Wide Transformations**: The data required to compute the records in a single partition may reside in many partitions of the parent RDD. 

Examples include:
* `distinct()`         
* `groupBy(..).sum()`  
* `sort(..)`           
* `repartition(n)`     

<img src="https://files.training.databricks.com/images/105/transformations-wide.png" alt="Wide Transformations" style="height:300px"/>

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Shuffles

A shuffle operation is triggered when data needs to move between executors.

For example, to group by color, it will serve us best if...
  * All the reds are in one partitions
  * All the blues are in a second partition
  * All the greens are in a third

From there we can easily sum/count/average all of the reds, blues, and greens.

To carry out the shuffle operation Spark needs to
* Convert the data to the UnsafeRow (if it isn't already), commonly referred to as Tungsten Binary Format.
* Write that data to disk on the local node - at this point the slot is free for the next task.
* Send that data across the wire to another executor
  * Technically the Driver decides which executor gets which piece of data.
  * Then the executor pulls the data it needs from the other executor's shuffle files.
* Copy the data back into RAM on the new executor
  * The concept, if not the action, is just like the initial read "every" `DataFrame` starts with.
  * The main difference being it's the 2nd+ stage.

As we will see in a moment, this amounts to a free cache from what is effectively temp files.

** *Note:* ** *Some actions induce in a shuffle.*<br/>
*Good examples would include the operations `count()` and `reduce(..)`.*

### UnsafeRow (aka Tungsten Binary Format)

As a quick side note, the data that is "shuffled" is in a format known as `UnsafeRow`, or more commonly, the Tungsten Binary Format.

`UnsafeRow` is the in-memory storage format for Spark SQL, DataFrames & Datasets. 

Advantages include:

* Compactness: 
  * Column values are encoded using custom encoders, not as JVM objects (as with RDDs). 
  * The benefit of using Spark 2.x's custom encoders is that you get almost the same compactness as Java serialization, but significantly faster encoding/decoding speeds. 
  * Also, for custom data types, it is possible to write custom encoders from scratch.

* Efficiency: Spark can operate _directly out of Tungsten_, without first deserializing Tungsten data into JVM objects.

-sandbox

<div style="text-align: center"><img src="https://files.training.databricks.com/images/unsafe-row-format.png"></div>

### How UnsafeRow works
* The first field, "123", is stored in place as its primitive.
* The next 2 fields, "data" and "bricks", are strings and are of variable length. 
* An offset for these two strings is stored in place (32L and 48L respectively shown in the picture below).
* The data stored in these two offset’s are of format “length + data”. 
* At offset 32L, we store 4 + "data" and likewise at offset 48L we store 6 + "bricks".

-sandbox
### Lightning-fast Serialization with Encoders
<div style="text-align:center"><img src="https://files.training.databricks.com/images/encoders-vs-serialization-benchmark.png"></div>

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Stages
* When we shuffle data, it creates what is known as a stage boundary.
* Stage boundaries represent a process bottleneck.

Take for example the following transformations:

|Step |Transformation|
|----:|--------------|
| 1   | Read    |
| 2   | Select  |
| 3   | Filter  |
| 4   | GroupBy |
| 5   | Select  |
| 6   | Filter  |
| 7   | Write   |

Spark will break this one job into two stages (steps 1-4b and steps 4c-8):

**Stage #1**

|Step |Transformation|
|----:|--------------|
| 1   | Read |
| 2   | Select |
| 3   | Filter |
| 4a | GroupBy 1/2 |
| 4b | shuffle write |

**Stage #2**

|Step |Transformation|
|----:|--------------|
| 4c | shuffle read |
| 4d | GroupBy  2/2 |
| 5   | Select |
| 6   | Filter |
| 7   | Write |

In **Stage #1**, Spark will create a pipeline of transformations in which the data is read into RAM (Step #1), and then perform steps #2, #3, #4a & #4b

All partitions must complete **Stage #1** before continuing to **Stage #2**
* It's not possible to group all records across all partitions until every task is completed.
* This is the point at which all the tasks must synchronize.
* This creates our bottleneck.
* Besides the bottleneck, this is also a significant performance hit: disk IO, network IO and more disk IO.

Once the data is shuffled, we can resume execution...

For **Stage #2**, Spark will again create a pipeline of transformations in which the shuffle data is read into RAM (Step #4c) and then perform transformations #4d, #5, #6 and finally the write action, step #7.

-sandbox
##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Pipelining
<img src="https://files.training.databricks.com/images/pipelining-2.png" style="float: right"/>

<ul>
  <li>Pipelining is the idea of executing as many operations as possible on a single partition of data.</li>
  <li>Once a single partition of data is read into RAM, Spark will combine as many narrow operations as it can into a single **Task**</li>
  <li>Wide operations force a shuffle, conclude, a stage and end a pipeline.</li>
  <li>Compare to MapReduce where: </li>
  <ol>
    <li>Data is read from disk</li>
    <li>A single transformation takes place</li>
    <li>Data is written to disk</li>
    <li>Repeat steps 1-3 until all transformations are completed</li>
  </ol>
  <li>By avoiding all the extra network and disk IO, Spark can easily out perform traditional MapReduce applications.</li>
</ul>

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Lineage
From the developer's perspective, we start with a read and conclude (in this case) with a write

|Step |Transformation|
|----:|--------------|
| 1   | Read    |
| 2   | Select  |
| 3   | Filter  |
| 4   | GroupBy |
| 5   | Select  |
| 6   | Filter  |
| 7   | Write   |
  
However, Spark starts with the action (`write(..)` in this case).

Next, it asks the question, what do I need to do first?

It then proceeds to determine which transformation precedes this step until it identifies the first transformation.

|Step |Transformation| |
|----:|--------------|-|
| 7   | Write   | Depends on #6 |
| 6   | Filter  | Depends on #5 |
| 5   | Select  | Depends on #4 |
| 4   | GroupBy | Depends on #3 |
| 3   | Filter  | Depends on #2 |
| 2   | Select  | Depends on #1 |
| 1   | Read    | First |

This would be equivalent to understanding your own lineage. 
* You don't ask if you are related to Genghis Khan and then work through the ancestry of all his children (5% of all people in Asia).
* You start with your mother.
* Then your grandmother
* Then your great-grandmother
* ... and so on
* Until you discover you are actually related to Catherine Parr, the last queen of Henry the VIII.

### Why Work Backwards?
**Question:** So what is the benefit of working backward through your action's lineage?<br/>
**Answer:** It allows Spark to determine if it is necessary to execute every transformation.

Take another look at our example:
* Say we've executed this once already
* On the first execution, step #4 resulted in a shuffle
* Those shuffle files are on the various executors (src & dst)
* Because the transformations are immutable, no aspect of our lineage can change.
* That means the results of our last shuffle (if still available) can be reused.

|Step |Transformation| |
|----:|--------------|-|
| 7   | Write   | Depends on #6 |
| 6   | Filter  | Depends on #5 |
| 5   | Select  | Depends on #4 |
| 4   | GroupBy | <<< shuffle |
| 3   | Filter  | *don't care* |
| 2   | Select  | *don't care* |
| 1   | Read    | *don't care* |

In this case, what we end up executing is only the operations from **Stage #2**.

This saves us the initial network read and all the transformations in **Stage #1**

|Step |Transformation|   |
|----:|---------------|:-:|
| 1   | Read          | *skipped* |
| 2   | Select        | *skipped* |
| 3   | Filter        | *skipped* |
| 4a  | GroupBy 1/2   | *skipped* |
| 4b  | shuffle write | *skipped* |
| 4c  | shuffle read  | - |
| 4d  | GroupBy  2/2  | - |
| 5   | Select        | - |
| 6   | Filter        | - |
| 7   | Write         | - |

### And Caching...

The reuse of shuffle files (aka our temp files) is just one example of Spark optimizing queries anywhere it can.

We cannot assume this will be available to us. 

Shuffle files are by definition temporary files and will eventually be removed.

However, we cache data to explicitly accomplish the same thing that happens inadvertently with shuffle files.

In this case, the lineage plays the same role. Take for example:

|Step |Transformation| |
|----:|--------------|-|
| 7   | Write   | Depends on #6 |
| 6   | Filter  | Depends on #5 |
| 5   | Select  | <<< cache |
| 4   | GroupBy | <<< shuffle files |
| 3   | Filter  | ? |
| 2   | Select  | ? |
| 1   | Read    | ? |

In this case we cached the result of the `select(..)`. 

We never even get to the part of the lineage that involves the shuffle, let alone **Stage #1**.

Instead, we pick up with the cache and resume execution from there:

|Step |Transformation|   |
|----:|---------------|:-:|
| 1   | Read          | *skipped* |
| 2   | Select        | *skipped* |
| 3   | Filter        | *skipped* |
| 4a  | GroupBy 1/2   | *skipped* |
| 4b  | shuffle write | *skipped* |
| 4c  | shuffle read  | *skipped* |
| 4d  | GroupBy  2/2  | *skipped* |
| 5a  | cache read    | - |
| 5b  | Select        | - |
| 6   | Filter        | - |
| 7   | Write         | - |

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Cleanup<br>

Run the **`Classroom-Cleanup`** cell below to remove any artifacts created by this lesson.

In [0]:
%run "../Includes/Classroom-Cleanup"

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/labs.png) Transformations & Actions Lab
It's time to put what we learned to practice.

Go ahead and open the notebook [Transformations And Actions Lab]($./Transformations And Actions Lab) and complete the exercises.

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>